In [41]:
import pandas as pd
from langchain.llms import OpenAI
# from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [32]:
tmp = pd.read_csv("lyrics_wf.csv")
tmp.head()

,URL,Lyrics,Title,Album Title
0,https://mojim.com/cny100019x82x1.htm,\n王菲\n\n\n\n\n\n\n归途有风\n\n\n电影 万里归途 主题曲\n作词：唐恬...,归途有风,归途有风
1,https://mojim.com/cny100019x81x1.htm,\n王菲\n\n\n\n\n\n\n如愿\n\n\n作词：唐恬\n作曲：钱雷\n你是 遥遥的...,如愿,如愿
2,https://mojim.com/cny100019x79x1.htm,\n王菲\n\n\n\n\n\n\n偶遇\n\n\n电影 邪不压正 宣传曲\n作词：林珺帆\...,偶遇,电影 邪不压正
3,https://mojim.com/cny100019x77x1.htm,\n王菲\n\n\n\n\n\n\n无问西东\n\n\n作词：彭青\n作曲：彭青\n编曲：彭...,无问西东,电影《无问西东》推广曲
4,https://mojim.com/cny100019x75x1.htm,\n王菲\n\n\n\n\n\n\n你在终点等我\n\n\n作词：姚若龙\n作曲：陈小霞\n...,你在终点等我,电影《从你的全世界路过》片尾曲


In [39]:
tmp.replace(r"\n{2,}", "\n", regex=True, inplace=True)
tmp.head()
new = tmp.assign(lyrics = tmp['Lyrics'].str.split("\n")).explode('lyrics')
new.head()


,URL,Lyrics,Title,Album Title,lyrics
0,https://mojim.com/cny100019x82x1.htm,\n王菲\n归途有风\n电影 万里归途 主题曲\n作词：唐恬@勇士音乐\n作曲：钱雷@勇士音...,归途有风,归途有风,
0,https://mojim.com/cny100019x82x1.htm,\n王菲\n归途有风\n电影 万里归途 主题曲\n作词：唐恬@勇士音乐\n作曲：钱雷@勇士音...,归途有风,归途有风,王菲
0,https://mojim.com/cny100019x82x1.htm,\n王菲\n归途有风\n电影 万里归途 主题曲\n作词：唐恬@勇士音乐\n作曲：钱雷@勇士音...,归途有风,归途有风,归途有风
0,https://mojim.com/cny100019x82x1.htm,\n王菲\n归途有风\n电影 万里归途 主题曲\n作词：唐恬@勇士音乐\n作曲：钱雷@勇士音...,归途有风,归途有风,电影 万里归途 主题曲
0,https://mojim.com/cny100019x82x1.htm,\n王菲\n归途有风\n电影 万里归途 主题曲\n作词：唐恬@勇士音乐\n作曲：钱雷@勇士音...,归途有风,归途有风,作词：唐恬@勇士音乐


In [42]:
# loader = CSVLoader(file_path="lyrics_wf.csv", source_column="Lyrics")
loader = DataFrameLoader(new, page_content_column='lyrics')
data = loader.load()

In [43]:
# Split documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Initialize OpenAI embeddings
embeddings_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings_model)

In [44]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [45]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [48]:
retriever=vectorstore.as_retriever()
lyrics = retriever.get_relevant_documents(
    query="今天好开心"
)

In [51]:
lyrics[0].page_content

'满心欢喜'

In [50]:
for lyric in lyrics:
    print("********--------*******")
    print(lyric)


********--------*******
page_content='满心欢喜' metadata={'URL': 'https://mojim.com/cny100019x60x20.htm', 'Lyrics': '\n王菲\n知己知彼\n作词：林夕\n作曲：Elisabeth Fraser / Robin Guthrie / Simon Raythonde\n编曲：Adrian Chan\n凌晨是我\n明晨是你\n事业陪着你\n衣服缠着我\n早已飞不起\n在两人的天地\n谁是我而谁是你\n唇是我而眉是你\n放于一起\n在这再难放置爱情的天地\n不知不觉爱已死\n共你知彼知己\n何必逃避\n红裙属我\n蓝筹属你\n玩物全属你\n饰物全属我\n更多更详尽歌词 在 ※ Mojim.com\u3000魔镜歌词网 \n一切花得起\n在这幸福天地\n谁是我而谁是你\n曾令我还曾令你\n满心欢喜\n在两个人各有各人的天地\n当初讲过爱到死\n就算今非昔比仍一起\n当初讲过爱到死\n就算今非昔比仍一起\n没患难但各自飞\n像天共地一般距离\n没患难但各自飞\n像天共地一般距离\n不知不觉爱已死\n共你知彼知己\n何必逃避\n当初讲过爱到死\n就算今非昔比仍一起\n', 'Title': '知己知彼', 'Album Title': '89-97 32首精选'}
********--------*******
page_content='然后这天' metadata={'URL': 'https://mojim.com/cny100019x60x24.htm', 'Lyrics': '\n王菲\n不再儿嬉\n心 为你在某天伤过\n在那日分手 你 讲过\n爱她更多\n当天 哭过 死过\u3000疯过 没结果\n当天你要我受折磨\n然后这天\n你却想再续旧日情\n愿再玩\n还道这天没有伴\n无新欢\nHey! 速速消失好嘛？\n不再儿嬉\n不相信动人话\n不要又来\n说梦话\n曾信梦话\n你现在\n完全是个笑话\n心 在某日某天好痛\n泪似浪汹涌\n那天你与她抱拥\n光阴使我知要醒觉及看通\n终于我抹掉你面容\n然后这夜你却宣调换立场\n说话漂亮\n求实我宽大地原谅\n重双双\nHey!\n更多更详尽歌词 在 ※ Mojim.com\u3000魔镜歌词网 \n速速消失好嘛？\